In [24]:
import os
orig_working_directory=os.getcwd()
orig_working_directory

'C:\\Users\\clemi\\Desktop\\Clementine-GWU-HW'

In [25]:
os.chdir(os.path.join('..'))
curr_working_directory=os.getcwd()
curr_working_directory

'C:\\Users\\clemi\\Desktop'

## Import Dependencies

In [26]:
import json
from sqlalchemy import create_engine
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

## Import configuration variables

In [27]:
from salesforce_config import sf_username, sf_password, sf_security_token
from salesforce_config import remote_db_endpoint, remote_db_port
from salesforce_config import remote_db_name, remote_db_user, remote_db_pwd

## Set up simple_salesforce

In [28]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

## Connect to your MySQL database

In [29]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

# Staff

### Query Data

In [30]:
staff_data = pd.read_sql("SELECT * FROM staff", conn)
staff_data.head(5)

,ID_Staff,EmployeeID,LastName,FirstName,MiddleName,BirthDate
0,1,000184220,Wimberly,Sam,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Popelka,Sarah,Nisan,1994-06-07


## Rename Columns

In [31]:
#staff_data['FullName'] = staff_data['FirstName'].str.cat(staff_data['LastName'], sep=" ")
staff_data=staff_data.rename(columns={"ID_Staff":"ID_Staff__c","EmployeeID":"EmployeeID__c","LastName":"Name",
                                       "FirstName":"FirstName__c","MiddleName":"MiddleName__c",
                                      "BirthDate":"BirthDate__c"})
#convert date to string otherwise we will get JSON nonserilizable error
staff_data["BirthDate__c"]=staff_data["BirthDate__c"].astype(str)
staff_data.head()

,ID_Staff__c,EmployeeID__c,Name,FirstName__c,MiddleName__c,BirthDate__c
0,1,000184220,Wimberly,Sam,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Popelka,Sarah,Nisan,1994-06-07


## Convert DataFrame to a list of Dictionaries

In [32]:
staff_load = staff_data.to_dict('records')
staff_load

[{'ID_Staff__c': 1,
  'EmployeeID__c': '000184220',
  'Name': 'Wimberly',
  'FirstName__c': 'Sam',
  'MiddleName__c': 'Nico',
  'BirthDate__c': '1995-07-05'},
 {'ID_Staff__c': 2,
  'EmployeeID__c': '000130109',
  'Name': 'Sanford',
  'FirstName__c': 'Gemini',
  'MiddleName__c': 'Blair',
  'BirthDate__c': '1992-04-22'},
 {'ID_Staff__c': 3,
  'EmployeeID__c': '000160655',
  'Name': 'Williams',
  'FirstName__c': 'Dartanion',
  'MiddleName__c': 'De Angelo',
  'BirthDate__c': '1993-05-21'},
 {'ID_Staff__c': 4,
  'EmployeeID__c': '000159108',
  'Name': 'Popelka',
  'FirstName__c': 'Sarah',
  'MiddleName__c': 'Nisan',
  'BirthDate__c': '1994-06-07'}]

## Inserting rows in bulk

In [33]:
try:
    sf.bulk.Staff__c.insert(staff_load)
except Exception as e:
    print(e)

# Staff Assignment

### Query Data 

In [34]:
staffassignment_data = pd.read_sql("SELECT * FROM staffassignment", conn)
staffassignment_data

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate
0,1,1,1,Teacher Assistant,2020-03-16,None
1,2,4,1,Teacher Assistant,2020-03-16,None
2,3,3,1,Instructor,2020-03-16,None


### Rename columns

In [35]:
staffassignment_data=staffassignment_data.rename(columns={"ID_StaffAssignment":"ID_StaffAssignment__c",
                                                          "ID_Staff":"ID_Staff__c", "ID_Class":"ID_Class__c","Role":"Name",
                                                           "StartDate":"StartDate__c","EndDate":"EndDate__c"})
#convert date to string otherwise we will get JSON nonserilizable error
staffassignment_data["StartDate__c"]=staffassignment_data["StartDate__c"].astype(str)

staffassignment_data.head()

,ID_StaffAssignment__c,ID_Staff__c,ID_Class__c,Name,StartDate__c,EndDate__c
0,1,1,1,Teacher Assistant,2020-03-16,None
1,2,4,1,Teacher Assistant,2020-03-16,None
2,3,3,1,Instructor,2020-03-16,None


### Extract Record ID 

In [36]:
query=sf.query("SELECT ID_Staff__c from Staff__c")

In [37]:
query

OrderedDict([('totalSize', 4),
             ('done', True),
             ('records',
              [OrderedDict([('attributes',
                             OrderedDict([('type', 'Staff__c'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Staff__c/a0C3h000000yWNyEAM')])),
                            ('ID_Staff__c', 1.0)]),
               OrderedDict([('attributes',
                             OrderedDict([('type', 'Staff__c'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Staff__c/a0C3h000000yWNzEAM')])),
                            ('ID_Staff__c', 2.0)]),
               OrderedDict([('attributes',
                             OrderedDict([('type', 'Staff__c'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Staff__c/a0C3h000000yWO0EAM')])),
                

In [38]:
staff_dict={}
x=0
while x< len(query['records']):
    url=query['records'][x]['attributes']['url'].split('/')[6]
    staff_id=int(query['records'][x]['ID_Staff__c'])
    print(url,staff_id)
    staff_dict[staff_id]=url
    x=x+1
print(staff_dict)

a0C3h000000yWNyEAM 1
a0C3h000000yWNzEAM 2
a0C3h000000yWO0EAM 3
a0C3h000000yWO1EAM 4
{1: 'a0C3h000000yWNyEAM', 2: 'a0C3h000000yWNzEAM', 3: 'a0C3h000000yWO0EAM', 4: 'a0C3h000000yWO1EAM'}


In [39]:
for index,row in staffassignment_data.iterrows():
    staffassignment_data['ID_Staff__c'][index]=staff_dict[row['ID_Staff__c']]
staffassignment_data

C:\Users\clemi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ID_StaffAssignment__c,ID_Staff__c,ID_Class__c,Name,StartDate__c,EndDate__c
0,1,a0C3h000000yWNyEAM,1,Teacher Assistant,2020-03-16,None
1,2,a0C3h000000yWO1EAM,1,Teacher Assistant,2020-03-16,None
2,3,a0C3h000000yWO0EAM,1,Instructor,2020-03-16,None


### Convert Dataframe to list of dictionaries

In [40]:
staffassignment_load = staffassignment_data.to_dict('records')
staffassignment_load 

[{'ID_StaffAssignment__c': 1,
  'ID_Staff__c': 'a0C3h000000yWNyEAM',
  'ID_Class__c': 1,
  'Name': 'Teacher Assistant',
  'StartDate__c': '2020-03-16',
  'EndDate__c': None},
 {'ID_StaffAssignment__c': 2,
  'ID_Staff__c': 'a0C3h000000yWO1EAM',
  'ID_Class__c': 1,
  'Name': 'Teacher Assistant',
  'StartDate__c': '2020-03-16',
  'EndDate__c': None},
 {'ID_StaffAssignment__c': 3,
  'ID_Staff__c': 'a0C3h000000yWO0EAM',
  'ID_Class__c': 1,
  'Name': 'Instructor',
  'StartDate__c': '2020-03-16',
  'EndDate__c': None}]

In [41]:
try:
    sf.bulk.StaffAssignment__c.insert(staffassignment_load)
except Exception as e:
    print(e)